In [ ]:
import pandas
import datetime
import matplotlib.pyplot as pyplot

# Data comparison with Rijkswaterstaat

Earlier analysis in the data shows that problems occur in the temperature sensor after the replacement of the door parts. The measurements flatline where they are expected to fluctuate with weather. In this document, research will be done by comparing the data from rijkswaterstaat with the data in our dataset.

In [ ]:
received_data = pandas.read_csv('data.csv')
rijkswaterstaat_data = pandas.read_csv('external.csv', delimiter=';')

In [ ]:
received_data['datetime'] = pandas.to_datetime(received_data['datetime'])

## Removing all other data

For this experiment, the water_temperatur is the only measurement of interest. All the other data will be dropped for the sake of consitency. The water_temperature_C column will also be renamed for clearity. This leaves the following columns. Undercarriage replacement can still be relevant considering the issues arrise after replacement of the part. There are also some outliers in the original data which shall be removed.

In [ ]:
droppable_columns = ['file_name', 'lock_movement', 'water_level_cmNAP', 'air_temperature_01C', 'energy_proxy']
renamable_columns = {'water_temperature_C': 'water_temperature'}

filtered_data = pandas.DataFrame(received_data)
filtered_data = filtered_data.drop(columns=droppable_columns)
filtered_data = filtered_data.rename(columns=renamable_columns)
filtered_data = filtered_data[filtered_data['water_temperature'] < 50]

filtered_data

The dataset received from rijkswaterstaat has a lot of columns describing the data. In order to get to a dataset useable for comparison, it is better to start with an empty data frame and add the columns needed, instead of removing everything not needed.

In [ ]:
rijkswaterstaat_data.head()

comparable_data = pandas.DataFrame()
comparable_data['date'] = rijkswaterstaat_data['WAARNEMINGDATUM']
comparable_data['time'] = rijkswaterstaat_data['WAARNEMINGTIJD (MET/CET)']
comparable_data['water_temperature'] = rijkswaterstaat_data['NUMERIEKEWAARDE']
comparable_data = comparable_data[::2]

comparable_data['date'] = pandas.to_datetime(comparable_data['date'], format="%d-%m-%Y").dt.date
comparable_data['time'] = pandas.to_datetime(comparable_data['time'], format="%H:%M:%S").dt.time

comparable_data

## Checking the problem

Before continueing, it is usefull to determine the actual problem we're trying to solve. The issue is some flatlining in watertemperature after the undercarriage got replaced. This can be seen in the graph below.

In [ ]:
filtered_data.plot.scatter('datetime', 'water_temperature')


This difference becomes more obvious when we display the data for only after the undercarriage replacement. The sensor seems to working fine at first, after which something happens to flatline the sensor.

In [ ]:
after_replacement = filtered_data[filtered_data['undercarriage_replacement'] == 1]
after_replacement.plot.scatter('datetime', 'water_temperature')

Since the drop in temperature goes to below 8 in the dataset, it is easy to determine the first moment the readings start to fail. In order to get all the values with a problem, all temperatures below 12 will be added to a new data frame. This way, we determine the first error to be at **2023-04-05**.

In [ ]:
problematic_data = after_replacement[after_replacement['water_temperature'] < 10]
problematic_data.head()

## Comparing with Rijkswaterstaat

Now it's time to put the claims to the test and start looking for inside the data of rijkswaterstaat whether this can aid our research. This is done by taking the date *2023-04-05* and filtering on that.

In [ ]:
ranged_data = pandas.DataFrame(comparable_data)
ranged_data['date'] = pandas.to_datetime(ranged_data['date'], format="%d-%m-%Y")
ranged_data['time'] = pandas.to_datetime(ranged_data['time'], format="%H:%M:%S")
ranged_data['time'] = ranged_data['time'].dt.time

ranged_data = ranged_data[ranged_data['date'] >= '05-04-2023']
ranged_data[ranged_data['time'] > datetime.time(10, 00, 00)]

## Making sure it fits

The data might be usable from a quality point of view, replacing the problematic values in the original data with good ones, however; there are some potential issues. We need to make sure that the data froms Rijkswaterstaat actually matches the data we have. If they differ too much, we might cause an unbalance in the results. The way to do this, is to compare two point of the sets we know are correct with each other.

In [ ]:
after_replacement.head()

In [ ]:
check_data = pandas.DataFrame(comparable_data)
check_data['date'] = pandas.to_datetime(check_data['date'], format="%d-%m-%Y")
check_data['time'] = pandas.to_datetime(check_data['time'], format="%H:%M:%S")
check_data['time'] = check_data['time'].dt.time

check_data = check_data[check_data['date'] >= '13-05-2022']
check_data[check_data['time'] > datetime.time(8, 30, 00)]

From this one instance, it can be determined that data is quite similar, precise enough for the replacement. This is how the issue with the data will be fought.

## Replacing the values

The last step in the research for this subject is to actually replace the incorrect values and to determine whether the problem is fixed. The only issue faced during this step is the fact that the original measurements times don't match the ones in the dataset. The plan is to round these times down during interpolation, since the difference of maximum 10 minutes won't effect the results too much.

In [ ]:
dated_problematic_data = pandas.DataFrame(problematic_data)
dated_problematic_data['datetime'] = pandas.to_datetime(dated_problematic_data['datetime'], format="%Y-%m-%d %H:%M:%S")

dated_problematic_data['date'] = dated_problematic_data['datetime'].dt.date
dated_problematic_data['floored_time'] = (dated_problematic_data['datetime'].dt.floor('10min')).dt.time

dated_problematic_data.head()

Now that a column is available containing the floored time, and one containing the date, the data can be matched. Trail and error baby let's go.

In [ ]:
merged_problematic_data = pandas.merge(dated_problematic_data, comparable_data, 
                                       left_on=['date', 'floored_time'], 
                                       right_on=['date', 'time'], 
                                       suffixes=('', '_y'))

merged_problematic_data['water_temperature'] = merged_problematic_data['water_temperature_y']
merged_problematic_data = merged_problematic_data.drop(columns=['floored_time', 'time', 'water_temperature_y', 'date'])
merged_problematic_data['water_temperature'] = merged_problematic_data['water_temperature'].str.replace(',', '.')
merged_problematic_data['water_temperature'] = merged_problematic_data['water_temperature'].astype(float)
merged_problematic_data.head()

In [ ]:
cleaned_data = pandas.merge(received_data, merged_problematic_data,
                            left_on=['datetime'],
                            right_on=['datetime'],
                            suffixes=('', '_y'),
                            how='left')

cleaned_data['water_temperature_C'] = cleaned_data['water_temperature'].where(cleaned_data['water_temperature'].notnull(), cleaned_data['water_temperature_C'])
cleaned_data = cleaned_data.drop(columns=['undercarriage_replacement_y', 'water_temperature'])
cleaned_data

In [ ]:
cleaned_data = cleaned_data[cleaned_data['water_temperature_C'] < 50]
cleaned_data.plot.scatter('datetime', 'water_temperature_C')

In [ ]:
index_verandering = cleaned_data[cleaned_data['undercarriage_replacement'] == 1].index[0]
cleaned_data['air_temperature_01C'] = cleaned_data['air_temperature_01C'] / 10

# Getting the date when the undercarriage was first replaced
datum_verandering = cleaned_data.loc[index_verandering, 'datetime']

# Setting the figure size
pyplot.figure(figsize=(12, 6))

# Plotting air temperature in blue
pyplot.scatter(cleaned_data.datetime, cleaned_data.air_temperature_01C, color='black', s=5, alpha=0.2, label='Lucht temperatuur')

# Plotting water temperature in green
pyplot.scatter(cleaned_data.datetime, cleaned_data.water_temperature_C, color='blue', s=5, alpha=0.2, label='Water temperatuur')

# Setting the title and labels
pyplot.title('Lucht- en watertemperatuur')
pyplot.xlabel('Datum')
pyplot.ylabel('Temperatuur (°C)')

# Rotating the x-axis labels for better readability
pyplot.xticks(rotation=45)

# Adding a grid
pyplot.grid(True)

# Adding a red vertical line to indicate the date of undercarriage replacement
pyplot.axvline(x=datum_verandering, color='red', label='Onderstel vervangen')

# Adding a legend
pyplot.legend()

# Displaying the plot
pyplot.show()

## Removing outliers

The next step is to remove the outliers in energy proxy

In [ ]:
cleaned_data.describe()

<br>
first, we split the data in before and after the undercarriage replacement

In [ ]:
cleaned_before_replacement = cleaned_data[cleaned_data.undercarriage_replacement ==  0]
cleaned_after_replacement = cleaned_data[cleaned_data.undercarriage_replacement ==  1]

In [ ]:
cleaned_before_replacement

In [ ]:
cleaned_after_replacement

next, we use the std and mean to remove the outliers <br>
here, std * 1.5 is used to differentiate between outliers and normal values. all values outside of the std * 1.5 + mean and std * 1.5 - mean, can be considered as outliers  

In [ ]:
outlier_threshold_before = 424007.2 * 1.5
mean_before = 3341879
cleaned_data_before = cleaned_before_replacement[(cleaned_before_replacement['energy_proxy'] <= mean_before +outlier_threshold_before) & (cleaned_before_replacement['energy_proxy'] >= mean_before - outlier_threshold_before)]

In [ ]:
cleaned_data_before

In [ ]:
outlier_threshold_after = 255254.2 * 1.5
mean_after = 2595676
cleaned_data_after = cleaned_after_replacement[(cleaned_after_replacement['energy_proxy'] <= mean_after + outlier_threshold_after) & (cleaned_after_replacement['energy_proxy'] >= mean_after - outlier_threshold_after)]

In [ ]:
cleaned_data_after

In [ ]:
before_corr = cleaned_data_before.select_dtypes(include=['float64', 'int64']).corr()
after_corr = cleaned_data_after.select_dtypes(include=['float64', 'int64']).corr()
total_corr = cleaned_data.select_dtypes(include=['float64', 'int64']).corr()

In [ ]:
import seaborn

pyplot.figure(figsize=(10, 8))  # Adjust width and height as needed

# Plot the heatmap
seaborn.heatmap(after_corr, annot=True, cmap='coolwarm')

# Show the plot
pyplot.show()

In [ ]:
cleaned_data_before.to_csv('cleaned_before.csv', index=False)

In [ ]:
cleaned_data_after.to_csv('cleaned_after.csv', index=False)

In [ ]:
cleaned_data.to_csv('restored.csv', index=False)